In [84]:
import pandas as pd
import csv, sys, re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn import tree, metrics
from sklearn.utils import shuffle

df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE , dtype= str, header = None, names=["instance", "text", "id", "sentiment","is_sarcastic"])

In [85]:
df=shuffle(df)
df.head(3)
df.id.value_counts()

10003    358
10000    244
10005    194
10006    189
10008    163
10001    140
10002    130
10015    119
10013    104
10016     59
10010     56
10019     52
10017     47
10018     38
10014     29
10012     25
10004     17
10009     16
10011     13
10007      7
Name: id, dtype: int64

In [86]:
text_data = np.array([])

In [87]:
for text in df.text:
    text_data = np.append(text_data,text)

In [88]:
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
    punctuations = '''!"&'()*+,-./:;<=>?[\]^`{|}~'''
    no_punct = ""
    for char in sample:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct


def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = remove_punctuation(sample)
    return sample


def myTokenizer(sample):
    """Customized tokenizer"""
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2]
    return new_words

In [89]:
count = CountVectorizer(preprocessor=myPreprocessor, lowercase = False, tokenizer=myTokenizer ,max_features = 200)
bag_of_words = count.fit_transform(text_data)
print(count.get_feature_names())

['#AFPRaids', '#AusPol', '#AusVotes', '#Auspol', '#Ausvotes', '#Brexit', '#Election2016', '#LNP', '#LNPfail', '#Labor', '#Medicare', '#NBN', '#Parakeelia', '#auspol', '#ausvotes', '#ausvotes2016', '#insiders', '#npc', '#qanda', '@AustralianLabor', '@LiberalAus', '@RichardDiNatale', '@TurnbullMalcolm', '@billshortenmp', 'AFP', 'ALP', 'Abbott', 'Australia', 'Australian', 'Australians', 'Bill', 'Coalition', 'Dutton', 'Government', 'Govt', 'Greens', 'How', 'If', 'Its', 'Joyce', 'LNP', 'Labor', 'Labors', 'Liberal', 'Liberals', 'Libs', 'Malcolm', 'Medicare', 'Morrison', 'NBN', 'No', 'Not', 'PM', 'Party', 'Peter', 'Shorten', 'So', 'THE', 'The', 'This', 'Turnbull', 'Turnbulls', 'VOTE', 'Vote', 'We', 'What', 'Why', 'You', 'about', 'after', 'again', 'all', 'amp', 'an', 'and', 'any', 'are', 'as', 'at', 'back', 'be', 'because', 'been', 'big', 'boats', 'budget', 'business', 'but', 'by', 'campaign', 'can', 'cant', 'care', 'could', 'cut', 'cuts', 'did', 'do', 'dont', 'down', 'economic', 'economy', 'e

In [90]:
print(count.vocabulary_)

{'has': 116, 'just': 129, 'for': 105, 'big': 83, 'Turnbull': 60, 'wants': 185, 'us': 181, 'to': 178, 'with': 193, 'amp': 72, 'Abbott': 26, 'because': 81, 'so': 163, '#ausvotes': 14, '#auspol': 13, 'PM': 52, 'Turnbulls': 61, 'of': 141, 'as': 77, '#NBN': 11, 'be': 80, 'must': 136, 'take': 165, 'and': 74, 'Govt': 34, 'was': 186, 'by': 88, 'LNP': 40, 'Bill': 30, 'Shorten': 55, 'its': 127, 'his': 120, 'own': 149, 'it': 126, 'says': 161, 'they': 174, 'NBN': 49, 'into': 124, 'We': 64, 'are': 76, 'all': 71, 'the': 170, 'election': 103, 'policy': 156, 'want': 184, 'from': 106, 'will': 192, 'do': 97, 'Australian': 28, 'on': 142, 'No': 50, 'in': 123, 'What': 65, 'you': 198, 'is': 125, 'our': 146, '#Election2016': 6, 'would': 197, 'not': 139, 'at': 78, 'taxes': 167, 'no': 138, 'Its': 38, '@TurnbullMalcolm': 22, 'budget': 85, 'he': 118, 'down': 99, 'ALP': 25, 'more': 135, 'who': 190, 'get': 109, 'tax': 166, 'cuts': 95, 'under': 179, 'Coalition': 31, 'Labor': 41, 'cant': 91, '#AusVotes': 2, 'one': 1

In [91]:
X = bag_of_words.toarray()

In [92]:
#target classes
Y = np.array([])
for text in df.id:
    Y = np.append(Y,text)

In [93]:
X_train = X[:1500]
X_test = X[1500:]
y_train = Y[:1500]
y_test = Y[1500:]

In [94]:
#clf = MultinomialNB()
#clf = BernoulliNB()
clf = tree.DecisionTreeClassifier(criterion='entropy',random_state=0) 
model = clf.fit(X_train, y_train)

In [95]:
predicted_y = model.predict(X_test)

In [96]:
print(accuracy_score(y_test, predicted_y))

0.252


In [97]:
print(y_test, predicted_y)
print(model.predict_proba(X_test))
print(accuracy_score(y_test, predicted_y))
print(precision_score(y_test, predicted_y, average='micro'))
print(recall_score(y_test, predicted_y, average='micro'))
print(f1_score(y_test, predicted_y, average='micro', labels = np.unique(predicted_y)))
print(f1_score(y_test, predicted_y, average='macro', labels = np.unique(predicted_y)))
print(classification_report(y_test, predicted_y,output_dict= False, labels = np.unique(predicted_y)))

['10011' '10003' '10016' '10002' '10001' '10004' '10005' '10013' '10013'
 '10000' '10004' '10003' '10003' '10016' '10006' '10005' '10003' '10000'
 '10001' '10000' '10001' '10001' '10001' '10000' '10002' '10018' '10006'
 '10005' '10000' '10015' '10000' '10000' '10000' '10003' '10004' '10006'
 '10008' '10006' '10015' '10006' '10005' '10015' '10000' '10005' '10008'
 '10003' '10010' '10000' '10000' '10006' '10002' '10015' '10010' '10016'
 '10003' '10013' '10003' '10003' '10019' '10004' '10019' '10013' '10016'
 '10008' '10008' '10003' '10003' '10008' '10000' '10015' '10001' '10017'
 '10000' '10005' '10015' '10018' '10003' '10010' '10008' '10005' '10000'
 '10002' '10000' '10001' '10003' '10008' '10013' '10008' '10001' '10001'
 '10006' '10013' '10003' '10002' '10003' '10003' '10015' '10003' '10018'
 '10002' '10008' '10005' '10010' '10006' '10002' '10000' '10017' '10008'
 '10005' '10019' '10013' '10003' '10005' '10015' '10001' '10005' '10005'
 '10001' '10006' '10000' '10005' '10008' '10005' '1

In [99]:
type(model.predict_proba(X_test))

numpy.ndarray

In [103]:
model.predict_proba(X_test)[4]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])